In [68]:
import flodym as fd
from flodym import ExcelDimensionReader, DimensionDefinition
from flodym import ExcelParameterReader, ParameterDefinition
import numpy as np
import flodym.export as fde
import pandas as pd

dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int),
    fd.DimensionDefinition(letter="c", name="time_construction", dtype=int),
    fd.DimensionDefinition(letter="b", name="building sector", dtype=str),
    fd.DimensionDefinition(letter="s", name="scenario", dtype=str),
    fd.DimensionDefinition(letter="p", name="product", dtype=str),
    fd.DimensionDefinition(letter="w", name="waste", dtype=str),
]

parameter_definitions = [
    fd.ParameterDefinition(name="building area per capita", dim_letters=("t","s")),
    fd.ParameterDefinition(name="population", dim_letters=("t",)),
    fd.ParameterDefinition(name="concrete mass per area", dim_letters=("t","p","s")),
    fd.ParameterDefinition(name="concrete mass per area historic", dim_letters=("c","p","s")),
    fd.ParameterDefinition(name="product share", dim_letters=("t","p")),
    fd.ParameterDefinition(name="stock product age cohort", dim_letters=("t","c","s","p")),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("c","s")),
    fd.ParameterDefinition(name="waste share", dim_letters=("c", "w")),
]

process_names = [
    "sysenv",
    "use",
    "waste collection", 
    "waste treatment"
]

flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="use", dim_letters=("c", "s", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste collection", dim_letters=("c", "s")),
    fd.FlowDefinition(from_process_name="waste collection", to_process_name="waste treatment", dim_letters=("c", "s", "w")),
]

stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("c", "s", "p"),
        time_letter="c",
        subclass=fd.StockDrivenDSM,
        lifetime_model_class=fd.NormalLifetime,
    ),
        fd.StockDefinition(
        name="waste treatment",
        process="waste treatment",
        dim_letters=("c", "s", "w"),
        time_letter="c",
        subclass=fd.SimpleFlowDrivenStock, # stock as the accumulation of inflow
    ),
]

mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

###Data import
dimension_file = "../Data/dimension.xlsx"
dimension_files = {d.name: dimension_file for d in dimension_definitions}
dimension_sheets = {d.name: d.name for d in dimension_definitions}
reader = ExcelDimensionReader(
    dimension_files=dimension_files,
    dimension_sheets=dimension_sheets,
)

dims = reader.read_dimensions(dimension_definitions)

parameter_file = "../Data/parameter_waste_t_to_c.xlsx"
parameter_files = {p.name: parameter_file for p in parameter_definitions}
parameter_sheets = {p.name: p.name for p in parameter_definitions}
reader = ExcelParameterReader(
    parameter_files=parameter_files,
    parameter_sheets=parameter_sheets,
    allow_missing_values=True,
    allow_extra_values=True,
)

parameters = reader.read_parameters(parameter_definitions=parameter_definitions, dims=dims)

In [69]:
class ConcreteMFA(fd.MFASystem):
    def compute(self):
        future_concrete_product_stock = self.parameters["population"] * self.parameters["building area per capita"] * \
                                        self.parameters["concrete mass per area"] * self.parameters["product share"]
        # print(f'future_concrete_product_stock: {future_concrete_product_stock}')



        # set the lifetime model parameters for stocks["use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.3 * parameters["product lifetimes"],  # assume std = 30% of mean
        )
        # build survival function for stocks["use"] using sf function (numpy array, ["c", "C", "s", "b", "p"])
        survival_historic_values = self.stocks["use"].lifetime_model.sf
        # print(type(survival_historic_values))
        # print(survival_historic_values.shape)

        # create FlodymArray survival_historic for survival function with correct dimensions
        c2 = fd.Dimension(name="time_construction2", letter="C", items=self.dims["c"].items)
        dims = fd.DimensionSet(dim_list=[c2] + self.stocks["use"].dims.dim_list)
        # as the stock composition is only available for 2015, slice the survival histroric to keep only C=2015
        survival_historic = fd.FlodymArray(values=survival_historic_values, dims=dims)[{"C": 2015}]

        # calculate the historical inflow from the stock composition and survival function for 2015
        # will get warning message for years later than 2015 where survival function = 0
        # but this will not affect the calculation as the stock after 2015 will be overwritten with predicted data later
        # this includes nan value
        inflow_historic = self.parameters["stock product age cohort"][{"t": 2015}] / survival_historic
        inflow_historic_values = inflow_historic.values
        inflow_historic_values_where_nan = np.isnan(inflow_historic_values)
        inflow_historic_values[inflow_historic_values_where_nan] = 0

        inflow_historic_nonan = fd.FlodymArray(values=inflow_historic_values, dims=inflow_historic.dims)*self.parameters["concrete mass per area historic"]

        # feed the historical inflow to an inflow-driven DSM
        dsm_historic = fd.InflowDrivenDSM(
            dims=inflow_historic_nonan.dims,
            lifetime_model=self.stocks["use"].lifetime_model,
            time_letter="c",
        )
        dsm_historic.inflow[...] = inflow_historic_nonan
        dsm_historic.compute()

        print(f'historic stock values from DSM: {dsm_historic.stock}')

        # change the use stock with historical stock data from DSM
        self.stocks["use"].stock[...] = dsm_historic.stock

        # overwrite future development with predicted stock
        self.stocks["use"].stock[{"c": self.dims["t"]}] = future_concrete_product_stock

        # print(f'future stock values: {self.stocks["use"].stock}')

        # compute use stock inflow and outflow
        self.stocks["use"].compute()

        use_outflow = self.stocks["use"].outflow
        # define envsys => use flow
        self.flows["sysenv => use"][...] = self.stocks["use"].inflow

        use_outflow_sum_p = use_outflow.sum_over(("p",))

        # current the dimension of use_outflow_t_slice_sum_p is 'T' and 's'
        print('--------')
        print(use_outflow_sum_p.dims)
        # need to change the dimension to 't' and 's'
        # use_outflow_t_slice_sum_p_Ttot = 

        self.flows["use => waste collection"][...] = use_outflow_sum_p
        waste_treatment_inflow = use_outflow_sum_p * self.parameters["waste share"]

        print(f'waste_treatment_inflow dimension is {waste_treatment_inflow}')

        self.flows["waste collection => waste treatment"][...] = waste_treatment_inflow
        

        self.stocks["waste treatment"].inflow[...] = waste_treatment_inflow
        self.stocks["waste treatment"].compute()

ConcreteMFA_FR = ConcreteMFA.from_excel(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
    dimension_sheets=dimension_sheets,
    parameter_sheets=parameter_sheets,
    allow_extra_parameter_values=True,
    allow_missing_parameter_values=True,
)


ConcreteMFA_FR.compute() #execute


historic stock values from DSM: StockArray 'unnamed_stock' with dims (c,s,p) and shape (201, 2, 17);
Values:
[[[    0.             0.             0.         ...     0.
       0.             0.        ]
  [    0.             0.             0.         ...     0.
       0.             0.        ]]

 [[    0.             0.             0.         ...     0.
       0.             0.        ]
  [    0.             0.             0.         ...     0.
       0.             0.        ]]

 [[    0.             0.             0.         ...     0.
       0.             0.        ]
  [    0.             0.             0.         ...     0.
       0.             0.        ]]

 ...

 [[    0.         75950.76930172     0.         ...  2710.85738259
   14728.50122023  9040.695948  ]
  [    0.         75950.76930172     0.         ...  2710.85738259
   14728.50122023  9040.695948  ]]

 [[    0.         71267.27965782     0.         ...  2527.78836711
   13799.32666218  8424.68489908]
  [    0.       

/Users/zhangyiwen/anaconda3/lib/python3.11/site-packages/flodym/flodym_arrays.py:349: RuntimeWarning:

divide by zero encountered in divide



In [70]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

ConcreteMFA_FR.check_mass_balance()
ConcreteMFA_FR.check_flows()

INFO:root:Checking mass balance of ConcreteMFA object...
INFO:root:Success - Mass balance is consistent!
INFO:root:Checking flows for NaN and negative values...
INFO:root:Success - No negative flows or NaN values in ConcreteMFA


In [71]:
ConcreteMFA_FR_flows_dict = ConcreteMFA_FR.flows
# print(ConcreteMFA_FR_flows_dict)
ConcreteMFA_FR_flows_collection_treatment = ConcreteMFA_FR_flows_dict['waste collection => waste treatment']

In [72]:
print(f'ConcreteMFA_FR_flows_collection_treatment dimensions are {ConcreteMFA_FR_flows_collection_treatment.dims.dim_list}')

ConcreteMFA_FR_flows_collection_treatment_df = ConcreteMFA_FR_flows_collection_treatment.to_df()

ConcreteMFA_FR_flows_collection_treatment dimensions are [Dimension(name='time_construction', letter='c', items=[1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047,

In [73]:
ConcreteMFA_FR_flows_collection_treatment_df.index

MultiIndex([(1900, 'BAU',                 'landfill'),
            (1900, 'BAU',              'backfilling'),
            (1900, 'BAU',                 'road use'),
            (1900, 'BAU',             'reuse_onsite'),
            (1900, 'BAU', 'recycle_no_recarbonation'),
            (1900, 'BAU',    'recycle_recarbonation'),
            (1900,  'CE',                 'landfill'),
            (1900,  'CE',              'backfilling'),
            (1900,  'CE',                 'road use'),
            (1900,  'CE',             'reuse_onsite'),
            ...
            (2100, 'BAU',                 'road use'),
            (2100, 'BAU',             'reuse_onsite'),
            (2100, 'BAU', 'recycle_no_recarbonation'),
            (2100, 'BAU',    'recycle_recarbonation'),
            (2100,  'CE',                 'landfill'),
            (2100,  'CE',              'backfilling'),
            (2100,  'CE',                 'road use'),
            (2100,  'CE',             'reuse_onsi

In [84]:
# change the multiindex to columns for analysis
ConcreteMFA_FR_flows_collection_treatment_table_df = ConcreteMFA_FR_flows_collection_treatment_df.reset_index()
ConcreteMFA_FR_flows_collection_treatment_table_df.columns = ['year', 'scenario', 'treatment', 'mass'] 


In [85]:
ConcreteMFA_FR_flows_collection_treatment_table_df

,year,scenario,treatment,mass
0,1900,BAU,landfill,0.000000
1,1900,BAU,backfilling,0.000000
2,1900,BAU,road use,0.000000
3,1900,BAU,reuse_onsite,0.000000
4,1900,BAU,recycle_no_recarbonation,0.000000
...,...,...,...,...
2407,2100,CE,backfilling,6412.809899
2408,2100,CE,road use,6412.809899
2409,2100,CE,reuse_onsite,6412.809899
2410,2100,CE,recycle_no_recarbonation,6412.809899


In [92]:
import pandas as pd

def aggregate_dataframe_by_time_period_filtered(
    df: pd.DataFrame, 
    aggregate_start_year: int, 
    aggregate_duration: int
) -> pd.DataFrame:
    """
    Aggregates the 'mass' column over specified time periods, 
    excluding all data prior to the aggregate_start_year.

    Args:
        df: The input DataFrame with columns: year, scenario, treatment, mass.
        aggregate_start_year: The year to start the first aggregation period (e.g., 2016).
        aggregate_duration: The length of each aggregation period in years (e.g., 10).

    Returns:
        A new DataFrame with aggregated data and columns: year_range, scenario, treatment, mass_range.
    """

    # 1. 🔍 FILTER the data to only include years >= aggregate_start_year
    df_filtered = df[df['year'] >= aggregate_start_year].copy()

    # 2. Calculate the start year of the aggregation block for each row
    # Example: year=2024, start=2016, duration=10
    # (2024 - 2016) // 10 = 8 // 10 = 0
    # 0 * 10 + 2016 = 2016 (The start year of the [2016-2025] block)
    df_filtered['agg_start_year'] = (
        (df_filtered['year'] - aggregate_start_year) // aggregate_duration
    ) * aggregate_duration + aggregate_start_year

    # 3. Calculate the 'year_range' string
    df_filtered['agg_end_year'] = df_filtered['agg_start_year'] + aggregate_duration - 1
    
    # Create the required range format (e.g., '2016-2025')
    df_filtered['year_range'] = (
        df_filtered['agg_start_year'].astype(str) + '-' + df_filtered['agg_end_year'].astype(str)
    )

    # 4. Group and Sum
    # Group by the new 'year_range', 'scenario', and 'treatment' and sum the 'mass'.
    aggregated_df = df_filtered.groupby(
        ['year_range', 'scenario', 'treatment'], 
        as_index=False
    )['mass'].sum()

    # 5. Rename and Finalize
    # Rename the aggregated 'mass' column to 'mass_range'.
    aggregated_df = aggregated_df.rename(columns={'mass': 'mass_range'})
    
    # Select the final desired columns
    return aggregated_df[['year_range', 'scenario', 'treatment', 'mass_range']]

In [93]:
ConcreteMFA_FR_flows_collection_treatment_year_agg_df = aggregate_dataframe_by_time_period_filtered(ConcreteMFA_FR_flows_collection_treatment_table_df, 2016, 10)

In [94]:
ConcreteMFA_FR_flows_collection_treatment_year_agg_df

,year_range,scenario,treatment,mass_range
0,2016-2025,BAU,backfilling,21981.300935
1,2016-2025,BAU,landfill,109906.504675
2,2016-2025,BAU,recycle_no_recarbonation,21981.300935
3,2016-2025,BAU,recycle_recarbonation,21981.300935
4,2016-2025,BAU,reuse_onsite,21981.300935
...,...,...,...,...
103,2096-2105,CE,landfill,161202.529321
104,2096-2105,CE,recycle_no_recarbonation,32240.505864
105,2096-2105,CE,recycle_recarbonation,32240.505864
106,2096-2105,CE,reuse_onsite,32240.505864


In [ ]:
import pandas as pd
ConcreteMFA_FR_flows_collection_treatment_2015_2100_df = ConcreteMFA_FR_flows_collection_treatment_table_df[ConcreteMFA_FR_flows_collection_treatment_table_df['year'] >= 2015].copy()


In [ ]:
fig_aggregated = fde.PlotlySankeyPlotter(mfa=ConcreteMFA_FR, exclude_processes=[]).plot()
fig_aggregated.show()

In [88]:
year_index = 2015
fig_one_year = fde.PlotlySankeyPlotter(mfa=ConcreteMFA_FR, exclude_processes=[], slice_dict = {"c": year_index}).plot()
fig_one_year.show()


In [ ]:
@Note: to ask the input to flow_color_dict arg
fig_by_waste = fde.PlotlySankeyPlotter(mfa=ConcreteMFA_FR, exclude_processes=[], 
                                       flow_color_dict = {'waste collection => waste treatment': ('waste',)}).plot()
fig_one_year.show()


ValidationError: 1 validation error for PlotlySankeyPlotter
  Value error, flow_color_dict must have a 'default' key to resort to if a flow is not in the dictionary [type=value_error, input_value={'mfa': ConcreteMFA(dims=...atment': ('landfill',)}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error